# 程序说明：
1. 对main2.py计算得到的节点位移和加速度结果进行统计分析和展示
   - 01

In [5]:
import os

# 指定 output文件路径
output_folder = os.path.join(os.getcwd(), "output")

# 获取文件夹中的所有文件名称
file_names = os.listdir(output_folder)

# 输出文件名称列表
print("output 文件夹中的文件名称:")
for file_name in file_names:
    print(file_name)

output 文件夹中的文件名称:
Model2_10yr_000
Model2_10yr_005
Model2_10yr_010
Model2_10yr_020
